In [1]:
import os
import copy
import glob
import shutil
import pandas as pd
import numpy as np
from scipy.io import savemat

import torch
from torch_geometric.utils import one_hot
from torch_scatter import scatter_add
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip

In [2]:
def create_df(csv_path):
        col_names = ['user_id', 'item_id', 'relation', 'ts']
        df = pd.read_csv(csv_path, sep='\t', names=col_names)
        df = df.drop('ts', axis=1)
        df_copy = df.copy()
        df['user_id'] = df['user_id'] - 1
        df['item_id'] = df['item_id'] - 1
        df['relation'] = df['relation'] - 1

        nums = {'user': df.max()['user_id'] + 1,
                'item': df.max()['item_id'] + 1,
                'node': df.max()['user_id'] + df.max()['item_id'] + 2,
                'edge': len(df)}
        return df_copy, nums

csv_path = "data\\ml-100k\\raw\\u1.base" # TODO: attach it somewhere 
user_item_df, nums = create_df(csv_path)
user_item_df.head()

,user_id,item_id,relation
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [3]:
# import files

def generate_item_genre_df(item_path, user_item_df):
    col_names = ['item_id', 'title', 'release_date', 'vid_release_date' ,'url']
    category_cols = ['unknown', 'action', 'adventure', 'animation' , 'children' , 'comedy' , 'crime' , 'documentary' , 'drama' , 'fantasy' ,'noir' , 'horror' , 'musical' , 'mystery' , 'romance' , 'scifi' ,'thriller' , 'war' , 'western']
    col = np.concatenate((col_names,category_cols))

    # extract data from csv
    item_genre_df = pd.read_csv(item_path, sep='|', names=col)
    col_length = len(item_genre_df.columns)
    
    # combine 2 dfs
    df = user_item_df.join(item_genre_df.set_index('item_id'), on='item_id') 
    df = df.drop(['user_id','item_id','relation','title','release_date','vid_release_date','url'], axis=1) # delete unwanted columns
    train_data = df.to_numpy()

    savemat('./data/ml-100k/processed/u1_base_genre.mat', mdict={'u1_genre_train': train_data, 'u1_genre_test': []}, do_compression=False)

    return train_data

item_path = "data\\ml-100k\\raw\\u.item" # TODO: attach it somewhere 
data = generate_item_genre_df(item_path,user_item_df)

In [119]:
torch.manual_seed(0)
scatter = torch.ones(10, 10).random_(2)
indexes = torch.nonzero(scatter)
print(len(indexes))

d_tensor = torch.tensor(data, dtype=torch.float32)
idx_data = torch.nonzero(d_tensor)
print(len(idx_data))

# total = torch.mul(scatter, d_tensor)
# idx_total = torch.nonzero(total)
# print(len(idx_total))

# last = (len(idx_total)/len(idx_data))*100
# last
# total

44
170437
